# Current Topics and Projects
## Sentiment Analysis - germansentiment

In [15]:
'''
This should be the 5th to be runned.
This file gets the sentiment by useing the german version of Bert.
This takes apprx. 2h
'''
import pandas as pd
from germansentiment import SentimentModel
from tqdm import tqdm

In [16]:
# df = pd.read_csv("comments_scores.csv")
df = pd.read_csv('comments_final.csv')
df_videos = pd.read_csv('videos.csv')

In [17]:
df_videos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   videoPublishedAt  153 non-null    object
 1   videolikeCount    153 non-null    int64 
 2   title             153 non-null    object
 3   description       153 non-null    object
 4   viewCount         153 non-null    int64 
 5   likeCount         153 non-null    int64 
 6   commentCount      153 non-null    int64 
 7   videochannelId    153 non-null    object
 8   videovideoID      153 non-null    object
 9   channelName       153 non-null    object
 10  transcript        153 non-null    object
dtypes: int64(4), object(7)
memory usage: 13.3+ KB


In [5]:
model = SentimentModel()

C:\Users\Raphaela\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Raphaela\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [18]:
total_entries = len(df)
with tqdm(total=total_entries, ncols=80) as pbar:
    def analyze_sentiment(row):
        global pbar

        classes, probabilities = model.predict_sentiment([row["text"]], output_probabilities=True)

        row["sentiment"] = classes[0]
        row["prob_pos"] = probabilities[0][0][1]
        row["prob_neg"] = probabilities[0][1][1]
        row["prob_neu"] = probabilities[0][2][1]

        pbar.update(1)  # Update the progress bar
        return row

    df = df.apply(analyze_sentiment, axis=1)

  0%|                                     | 5/154122 [00:04<40:21:22,  1.06it/s]


KeyboardInterrupt: 

In [19]:
# same for transcript of videos

total_entries = len(df_videos)
with tqdm(total=total_entries, ncols=80) as pbar:
    def analyze_sentiment(row):
        global pbar

        classes, probabilities = model.predict_sentiment([row["transcript"]], output_probabilities=True)

        row["sentiment_transcript"] = classes[0]
        row["prob_pos_transcript"] = probabilities[0][0][1]
        row["prob_neg_transcript"] = probabilities[0][1][1]
        row["prob_neu_transcript"] = probabilities[0][2][1]

        pbar.update(1)  # Update the progress bar
        return row

    df_videos = df_videos.apply(analyze_sentiment, axis=1)

100%|█████████████████████████████████████████| 153/153 [10:20<00:00,  4.06s/it]


In [21]:
df_videos["prob_neg_transcript"].describe()

count    153.000000
mean       0.102240
std        0.206582
min        0.006049
25%        0.020132
50%        0.029650
75%        0.053154
max        0.980246
Name: prob_neg_transcript, dtype: float64

In [23]:
df.to_csv("comments_scores.csv", index = False)
df_videos.to_csv("videos.csv", index = False)